In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
construido = pd.read_parquet("construido.parquet")
educacion = pd.read_parquet("education.parquet")
income = pd.read_parquet("income.parquet")
over65 = pd.read_parquet("over65.parquet")
poblacion = pd.read_parquet("poblacion.parquet")
renta = pd.read_parquet("renta.parquet")
turismo = pd.read_parquet("tourism.parquet")
waste = pd.read_parquet("waste.parquet")

In [27]:
years = list(waste.YEA.unique())
years.sort()
print(years)

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
